In [11]:
from pyspark.sql.functions import split, explode, col, regexp_replace
from pyspark.sql import SparkSession
import pyspark
import collections
import re
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField

# Crear una sesión de Spark
spark = SparkSession.builder.appName("EjemploPySpark").getOrCreate()

# Ruta al archivo CSV
file = 'data_user_reels_claudianicolasa.csv'

lines = spark.sparkContext.textFile(file)

'''
El csv tiene el problema de que un campo esta ocupando una o varias filas del csv
es_incompleta y unir_lineas_incompletas son dos metodos para solventar este problema
es_incompleta para comprobar si el campo usa mas de una linea
unir_lineas_incompletas para joinear esas lineas
'''

# Función para eliminar comas dentro de comillas dobles en una línea CSV
def eliminar_comas_entre_comillas(line):
    # Usamos una expresión regular que encuentra cualquier cosa entre comillas dobles
    # y luego eliminamos las comas que están dentro de ese texto
    return re.sub(r'\"(.*?)\"', lambda match: match.group(0).replace(",", ""), line)


# Función para verificar si la línea está incompleta
def es_incompleta(line):
    # Verifica si la línea contiene un número impar de comillas dobles
    return line.count('"') % 2 != 0

# Acumulamos las líneas hasta que tengamos una línea completa (con un número par de comillas)
def unir_lineas_incompletas(rdd):
    acumulado = []
    resultados = []

    for line in rdd.collect():
        acumulado.append(line)
        # Si la línea completa el conjunto de comillas
        if not es_incompleta(" ".join(acumulado)):
            # Unimos las líneas acumuladas y las añadimos al resultado
            resultados.append(" ".join(acumulado))
            acumulado = []
    
    return resultados

# Unir las líneas mal formadas en el RDD
lineas_corregidas = spark.sparkContext.parallelize(unir_lineas_incompletas(lines))


# Aplicamos la función a cada línea corregida para eliminar las comas entre comillas
rdd_sin_comas = lineas_corregidas.map(eliminar_comas_entre_comillas)

# Ahora convertimos cada línea en una lista de columnas usando split por comas
rdd_estructurado = rdd_sin_comas.map(lambda line: line.split(","))


# Creamos el schema del dataframe

esquema = StructType([
    StructField("id", IntegerType(), True),
    StructField("comentario", IntegerType(), True),
    StructField("like", IntegerType(), True),
    StructField("fecha", DateType(), True),
    StructField("tipopublicacion", IntegerType(), True),
    StructField("oldvisualizaciones", IntegerType(), True),
    StructField("visualizaciones", IntegerType(), True),
    StructField("titulo", StringType(), False)
])

     
# Convertimos el RDD en DataFrame usando toDF y asignamos nombres de columnas
df = spark.createDataFrame(rdd_estructurado, schema=esquema)

df.printSchema()

df = df.withColumn("totalViews", df.oldvisualizaciones + df.visualizaciones)

#df.show(10)

#df = df.withColumn("titulopublicacion", split(df["titulo"], "#").getItem(0)) \
#       .withColumn("hastag", split(df["titulo"], "#").getItem(1))







root
 |-- id: integer (nullable = true)
 |-- comentario: integer (nullable = true)
 |-- like: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- tipopublicacion: integer (nullable = true)
 |-- oldvisualizaciones: integer (nullable = true)
 |-- visualizaciones: integer (nullable = true)
 |-- titulo: string (nullable = false)

